<a href="https://colab.research.google.com/github/Shritej24c/Trading-Bot-RL/blob/Avinash/FinRL_Package.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-reflpyet
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-reflpyet
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-fksjpp0i/pyfolio_7c5ce8f1d0734147840f7afc9e724882
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-fksjpp0i/pyfolio_7c5ce8f1d0734147840f7afc9e724882
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-fksjpp0i/elegantrl_c24c846cf4564fe0bc431ca0dd1d21ab
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-fksjpp0i/elegantrl_c24c846cf4564fe0bc431ca0dd1d21ab


In [ ]:
import finrl

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
matplotlib.use('Agg')
import datetime
import os
from finrl import config
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent
from finrl.trade.backtest import BackTestStats, BaselineStats, BackTestPlot
import sys
sys.path.append("../FinRL-Library")

ModuleNotFoundError: ignored

In [ ]:
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-va8ymu46
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-va8ymu46
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-wcmc6fjm/pyfolio_ce854c490074486faac6270d07ff546b
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-wcmc6fjm/pyfolio_ce854c490074486faac6270d07ff546b
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-wcmc6fjm/elegantrl_1cf3d1ecb8f0403683761e97e92a5e4c
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-wcmc6fjm/elegantrl_1cf3d1ecb8f0403683761e97e92a5e4c


In [ ]:
import finrl.config

In [ ]:
from finrl import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent
from finrl.trade.backtest import BackTestStats, BaselineStats, BackTestPlot
import sys
sys.path.append("../FinRL-Library")

ImportError: ignored

In [ ]:
data_df = YahooDownloader(start_date = '2009-01-01',
                          end_date = '2021-01-01',
                          ticker_list = ['AAPL']).fetch_data()
data_df.head()